In [1]:
import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import copy
from typing import Literal
import argparse
import pandas as pd
import numpy as np
import os

import torch
from torch import nn
from torch.distributions import Distribution
from torch_uncertainty.utils.distributions import cat_dist
from torch_uncertainty.routines import ClassificationRoutine
from torch_uncertainty.utils import TUTrainer
from torch_uncertainty.models import deep_ensembles
from torch_uncertainty.transforms import RepeatTarget
import torchvision.transforms as T

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

In [2]:
### load the dataset

results_dict = {}

labeling_functions=[
    "guo_los",
    "guo_readmission",
    "guo_icu",
    "new_hypertension",
    "new_hyperlipidemia",
    "new_pancan",
    "new_celiac",
    "new_lupus",
    "new_acutemi",
    "lab_thrombocytopenia",
    "lab_hyperkalemia",
    "lab_hyponatremia",
    "lab_anemia",
    "lab_hypoglycemia" # will OOM at 200G on `gpu` partition
]


In [3]:
max_epochs = 50
batch_size = 64
task = labeling_functions[0]
folder_path = f'single_task_data/{task}'

train_x_name = os.path.join(folder_path, 'X_train.csv')
train_y_name = os.path.join(folder_path, 'y_train.csv')
val_x_name = os.path.join(folder_path, 'X_val.csv')
val_y_name = os.path.join(folder_path, 'y_val.csv')
test_x_name = os.path.join(folder_path, 'X_test.csv')
test_y_name = os.path.join(folder_path, 'y_test.csv')

X_train = pd.read_csv(train_x_name).to_numpy()
y_train = pd.read_csv(train_y_name).to_numpy().reshape(-1)
X_val = pd.read_csv(val_x_name).to_numpy()
y_val = pd.read_csv(val_y_name).to_numpy().reshape(-1)
X_test = pd.read_csv(test_x_name).to_numpy()
y_test = pd.read_csv(test_y_name).to_numpy().reshape(-1)

# Create class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float)

X_train = torch.tensor(X_train).float()
X_val = torch.tensor(X_val).float()
X_test = torch.tensor(X_test).float()
y_train = torch.tensor(y_train).long()
y_val = torch.tensor(y_val).long()
y_test = torch.tensor(y_test).long()

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TwoLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


input_size = X_train.shape[1]
hidden_size = 128
num_classes = 2
model = TwoLayerNet(input_size, hidden_size, num_classes)

In [5]:
def optim_recipe(model, lr_mult: float = 1.0):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05 * lr_mult)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    return {"optimizer": optimizer, "scheduler": scheduler}

In [6]:
# Create the ensemble model
ensemble = deep_ensembles(
    model,
    num_estimators=5,
    task="classification",
    reset_model_parameters=True,
)

trainer = TUTrainer(accelerator="gpu", max_epochs=max_epochs)
ens_routine = ClassificationRoutine(
    is_ensemble=True,
    num_classes=2,
    model=ensemble,
    loss=nn.CrossEntropyLoss(weight=class_weights),
    format_batch_fn=RepeatTarget(
        5
    ),  # How to handle the targets when comparing the predictions
    optim_recipe=optim_recipe(
        ensemble, 2.0
    ),  # The optimization scheme with the optimizer and the scheduler as a dictionnary
    eval_ood=False,  # We want to evaluate the OOD-related metrics
)
trainer.fit(ens_routine, train_dataloaders=train_dl, val_dataloaders=test_dl)
ens_perf = trainer.test(ens_routine, dataloaders=[test_dl])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | _DeepEnsembles   | 493 K  | train
1 | loss                | CrossEntropyLoss | 0      | train
2 | format_batch_fn     | RepeatTarget     | 0      | train
3 | val_cls_metrics     | MetricCollection | 0      | train
4 | test_cls_metrics    | MetricCollection | 0      | train
5 | test_id_entro

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (41) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │          0.13276          │
│       test/cal/aECE       │          0.12849          │
│       test/cls/Acc        │          0.77813          │
│      test/cls/Brier       │          0.34285          │
│       test/cls/NLL        │          0.63426          │
│     test/cls/entropy      │          0.21674          │
│   test/ens_Disagreement   │          0.06979          │
│     test/ens_Entropy      │          0.19437          │
│        test/ens_MI        │          0.02237          │
│       test/sc/AUGRC       │          0.06650          │
│       test/sc/AURC        │          0.09467          │
│    test/sc/CovAt5Risk     │            nan            │
│    test/sc/RiskAt80Cov    │          0.16800          │
└───────────────────────────┴───────────────────────────┘

[{'test/cal/ECE': 0.13276050984859467,
  'test/cal/aECE': 0.12848876416683197,
  'test/cls/Acc': 0.7781321406364441,
  'test/cls/Brier': 0.3428516387939453,
  'test/cls/NLL': 0.6342589259147644,
  'test/sc/AUGRC': 0.06650328636169434,
  'test/sc/AURC': 0.0946667343378067,
  'test/sc/CovAt5Risk': nan,
  'test/sc/RiskAt80Cov': 0.16799543797969818,
  'test/cls/entropy': 0.21673902869224548,
  'test/ens_Disagreement': 0.06979498267173767,
  'test/ens_Entropy': 0.19437210261821747,
  'test/ens_MI': 0.022366950288414955}]

: 